In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
#import src
import Cleanup
import Transformation
import FileProcessing

In [3]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [4]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [5]:
cleanup.remove_dups(ngsim)

704000 duplicate values have been removed


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [ ]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [ ]:
ngsim.shape

(11146526, 25)

In [ ]:
ngsim=transform.drop_not_required_columns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [ ]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.create_column_placeholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [ ]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [ ]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [ ]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [ ]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [ ]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

Vehicle Combination of the Preceding and the Following Vehicle. 

In [ ]:
filtered_ngsim_U=transform.map_previous_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_previous_vehicle_details(filtered_ngsim_I)

0          us-101
3          us-101
4          us-101
6          us-101
7          us-101
            ...  
8704795    us-101
8704797    us-101
8704798    us-101
8704799    us-101
8704800    us-101
Name: Location, Length: 4098933, dtype: object::621 cars dont change lanes
0          us-101
3          us-101
4          us-101
6          us-101
7          us-101
            ...  
8704795    us-101
8704797    us-101
8704798    us-101
8704799    us-101
8704800    us-101
Name: Location, Length: 4098933, dtype: object::2226 cars Change lanes
2           i-80
10          i-80
12          i-80
13          i-80
14          i-80
            ... 
11850521    i-80
11850522    i-80
11850523    i-80
11850524    i-80
11850525    i-80
Name: Location, Length: 4566387, dtype: object::817 cars dont change lanes
2           i-80
10          i-80
12          i-80
13          i-80
14          i-80
            ... 
11850521    i-80
11850522    i-80
11850523    i-80
11850524    i-80
11850525    i-80
Name: Loc

In [ ]:
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)

float64
float64
dataset before Row Removal(4098933, 33)
(719729, 33) Lead and Subject both change lane, so first and last 5 seconds of trajectory removed
(36238, 33) Lead Vehicle changes Lane, so first 5 seconds of car following Removed. 
         Vehicle_ID  Frame_ID    Global_Time  Local_X   Local_Y     Global_X  \
60               14        32  1118846982100   49.079    49.741  6451122.979   
61               14        33  1118846982200   49.059    53.742  6451125.668   
68               14        34  1118846982300   49.037    57.741  6451128.357   
72               14        35  1118846982400   49.017    61.741  6451131.045   
80               14        36  1118846982500   48.997    65.742  6451133.734   
...             ...       ...            ...      ...       ...          ...   
4098193        1945      9828  1118849739800   54.446  2143.291  6452659.035   
4098206        1945      9829  1118849739900   54.443  2147.803  6452662.556   
4098228        1945      9830  1118849740


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [ ]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)

fileProcessing.export_file(filtered_ngsim,'Cleaned_NGSIM_Data')

1915446
1945651
3861097


True

In [ ]:
print(f"{filtered_ngsim[(filtered_ngsim['previous_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['previous_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

2387 cars Change lanes
2484 cars DonT


In [ ]:
#filtered_ngsim[((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('classVERIFYFILE.CSV', index = False)

In [ ]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Relative_Time,Vehicle_combination,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,previous_Vehicle_Velocity,previous_Vehicle_Acceleration,previous_car_lane_changes,pair_Time_Duration,total_pair_duration
0,5,8,1118846979700,39.788,39.154,6451122.815,1873326.569,5.18160,2,12.1920,...,1117683960601,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.0,160.8
1,5,9,1118846979800,39.767,43.153,6451125.503,1873323.608,5.18160,2,12.1920,...,1117683960701,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.1,160.8
2,5,10,1118846979900,39.747,47.154,6451128.192,1873320.646,5.18160,2,12.1920,...,1117683960801,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.2,160.8
3,5,11,1118846980000,39.726,51.154,6451130.881,1873317.684,5.18160,2,12.1920,...,1117683960901,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.3,160.8
4,5,12,1118846980100,39.705,55.153,6451133.569,1873314.723,5.18160,2,12.1920,...,1117683961001,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.4,160.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566239,2485,11599,1113438724800,54.755,1595.253,6042663.224,2134655.863,4.20624,2,0.0762,...,1112275705701,Car-Car,4.52628,True,2485,0.0,0.0,False,122.9,128.4
4566241,2485,11600,1113438724900,54.755,1595.278,6042663.220,2134655.888,4.20624,2,0.0762,...,1112275705801,Car-Car,4.52628,True,2485,0.0,0.0,False,123.0,128.4
4566242,2485,11601,1113438725000,54.755,1595.303,6042663.216,2134655.912,4.20624,2,0.0762,...,1112275705901,Car-Car,4.52628,True,2485,0.0,0.0,False,123.1,128.4
4566245,2485,11602,1113438725100,54.756,1595.328,6042663.213,2134655.937,4.20624,2,0.0762,...,1112275706001,Car-Car,4.52628,True,2485,0.0,0.0,False,123.2,128.4


In [ ]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Relative_Time,Vehicle_combination,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,previous_Vehicle_Velocity,previous_Vehicle_Acceleration,previous_car_lane_changes,pair_Time_Duration,total_pair_duration
1113491,2964,8731,1118847852000,4.868,309.729,6451336.355,1873154.258,5.02920,2,2.618232,...,1117684832901,Car-Car,4.72440,False,2964,0.0000,0.0,False,29.7,89.6
2553891,2491,7916,1113437558500,17.161,598.884,6042774.601,2133664.163,4.66344,2,3.453384,...,1112274539401,Car-Car,4.51104,True,2491,3.5814,0.0,False,33.0,99.8
